Il seguente script crea le cartelle (dopo aver controllato che non esistano già) "audio", "docs" e "images".Dopodiché, itera la cartella principale spostando ogni file a seconda della propria estensione nella cartella di riferimento e compila un file csv in cui annota il nome, il formato e la dimensione di tale file. Come da indicazione, si suppone che il notebook sia allo stello livello(cioè nella stessa cartella) della cartella files che contiene i file da spostare. Si utilizzano pertanto riferimenti relativi

In [1]:
# Script step 1

# importiamo i moduli che ci serviranno
import os
import shutil
import csv

# per prima cosa, creiamo le cartelle (se non esistono già) in cui verranno spostati i file.
for foldername in ["audio", "docs", "images"]:
    new_folder = os.path.join("", foldername)
    if not os.path.exists(new_folder):
        os.makedirs(new_folder)

# creiamo le tuple che contengono i formati file più comuni per ogni tipo di file che andremo ad analizzare:

docs_extensions = (".txt", ".doc", ".docx", ".pages", ".odt", ".rtf", ".tex")
audio_extensions = (".mp3", ".mp4", ".m4a", ".wma", ".flac", ",aac")
images_extensions = (".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff")

# controllo se il file di recape esiste ed eventualmente lo creo
if not os.path.exists("recap.csv"):  # se non esiste
    recap = open("recap.csv","w", newline="")# lo creo
    writer = csv.writer(recap)
    writer.writerow(["name", "type", "size (B)"]) # scrivo header del csv
else:  # se esiste già
    recap = open("recap.csv", "a", newline="") # lo apro in 'append' con 'a'
    writer = csv.writer(recap)


# iteriamo in ordine alfabetico i file nella cartella "files":
for file in sorted(os.listdir()):
    
    file_name, file_extention = os.path.splitext(file) # separo il nome del file dalla sua estensione
    current_folder = "" + file  # specifica il percorso del file
    size = os.stat(current_folder).st_size  # calcola la dimensione del file

    # usiamo il metodo "endswith" per prendere in considerazione i formati file e smistarli nelle relative cartelle:
    if file.endswith(docs_extensions):
        # questa sarà la cartella in cui verrà spostato il file docs
        final_folder = ("docs/" + file)
        # con shutil.move sposto il file dalla cartella iniziale a quella di destinazione
        shutil.move(current_folder, final_folder)
        writer.writerow([file_name, "doc", size])  # aggiorno recap
        # stampo le info richieste dei file(nome, tipo e dimensioni in byte) secondo l'output desiderato
        print("{} type:doc size:{}B".format(file_name, size))

    elif file.endswith(audio_extensions):
        # questa sarà la cartella in cui verrà spostato il file audio
        final_folder = ("audio/" + file)
        # con shutil.move sposto il file dalla cartella iniziale a quella di destinazione
        shutil.move(current_folder, final_folder)
        writer.writerow([file_name, "audio", size])  # aggiorno recap
        # stampo le info richieste dei file(nome, tipo e dimensioni in byte) secondo l'output desiderato
        print("{} type:audio size:{}B".format(file_name, size))

    elif file.endswith(images_extensions):
        # questa sarà la cartella in cui verrà spostato il file image
        final_folder = ("images/" + file)
        # con shutil.move sposto il file dalla cartella iniziale a quella di destinazione
        shutil.move(current_folder, final_folder)
        writer.writerow([file_name, "image", size])  # aggiorno recap
        # stampo le info richieste dei file(nome, tipo e dimensioni in byte) secondo l'output desiderato
        print("{} type:image size:{}B".format(file_name, size))

recap.close()  # chiudo il file di recap al termine della compilazione

bw type:image size:94926B
ciao type:doc size:12B
daffodil type:image size:24657B
eclipse type:image size:64243B
pippo type:doc size:8299B
song1 type:audio size:1087849B
song2 type:audio size:764176B
trump type:image size:10195B


Il seguente script, invece, analizza le immagini presenti nella cartella images e ne estrae le informazioni di colore sfruttando la libreria Numpy.
Infine stampa una tabella contenente le informazioni raccolte.

In [2]:
# Script step 3

# importiamo i moduli che ci serviranno
import os
import numpy as np
from PIL import Image
from tabulate import tabulate

# creiamo una tabella vuota che verrà riempita ad ogni iterazione
tab = []

# iteriamo i file nella cartella "images"
for im in os.listdir("images"):
    image_file = ("images/" + im) # specifica il percorso del file
    image = Image.open(image_file)  # apre l'immagine
    image_array = np.array(image)  # trasforma l'immagine in un array numpy
    name = im.split(".")[0]  # separa il nome dal formato per isolare il nome
    height = np.shape(image_array)[0]  # restituisce l'altezza dell'immagine
    width = np.shape(image_array)[1]  # restituisce la larghezza dell'immagine

# usiamo "ndim" per effettuare diverse operazioni a seconda del numero delle dimensioni dell'array
    if image_array.ndim == 2:  # se le dimensioni sono 2, vuol dire che si tratta di un'immagine in bianco e nero
        grayscale = np.mean(image_array)  # calcoliamo la scala del grigio
        R, G, B, ALPHA = 0.00, 0.00, 0.00, 0.00

    elif image_array.ndim == 3:  # se le dimensioni sono 3, vuol dire che si tratta di un'immagine a colori
        grayscale = 0.00  # essendo un'immagine a colori, la scala di grigio sarà 0
        # con l'uso degli slicing degli array calcoliamo la scala di colori facendone la media
        R = np.mean(image_array[:, :, 0])
        G = np.mean(image_array[:, :, 1])
        B = np.mean(image_array[:, :, 2])

        # se gli elementi della terza dimensione sono 3, l'immagine è RGB, quindi ALPHA sarà 0
        if np.shape(image_array)[2] == 3:
            ALPHA = 0.00

        # se gli elementi della terza dimensione sono 4, l'immagine è RGBA, quindi andremo a calcolare il valore di ALPHA
        elif np.shape(image_array)[2] == 4:
            ALPHA = np.mean(image_array[:, :, 3])

# creiamo una lista con le informazioni ottenute e l'aggiungiamo alla lista "tabella", creando così una lista di liste
    tab.append([name, height, width, grayscale, R, G, B, ALPHA])

# infine con il modulo "tabulate" creiamo e stampiamo la tabella, specificande nell'headers le etichette delle colonne
headers = ["name", "height", "width", "grayscale", "R", "G", "B", "ALPHA"]
print(tabulate(tab, headers=headers,tablefmt="fancy_grid",floatfmt='.2f'))

╒══════════╤══════════╤═════════╤═════════════╤════════╤════════╤═══════╤═════════╕
│ name     │   height │   width │   grayscale │      R │      G │     B │   ALPHA │
╞══════════╪══════════╪═════════╪═════════════╪════════╪════════╪═══════╪═════════╡
│ bw       │      512 │     512 │       21.48 │   0.00 │   0.00 │  0.00 │    0.00 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ daffodil │      500 │     335 │        0.00 │ 109.25 │  85.56 │  4.97 │    0.00 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ eclipse  │      256 │     256 │        0.00 │ 109.05 │ 109.52 │ 39.85 │  133.59 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ trump    │      183 │     275 │        0.00 │  97.01 │  98.99 │ 90.92 │    0.00 │
╘══════════╧══════════╧═════════╧═════════════╧════════╧════════╧═══════╧═════════╛
